# Deploy SquadStack Speech Recognition (Hinglish) Model Package from AWS Marketplace 


---

This notebook's CI test result for ap-south-1 is as follows.

---

Meet SquadStack ASR, a groundbreaking speech-to-text solution designed for both technology enthusiasts and business users. This innovative AI model boasts top-notch Word Error Rate (WER) performance, ensuring highly accurate transcriptions. SquadStack ASR also offers breakthrough performance in Hinglish transcription, making it perfect for mixed-language environments. With its swift processing capabilities, SquadStack ASR effectively saves time and enhances productivity. Best of all, our competitively priced solution delivers exceptional value without compromising quality. Embrace the future of speech recognition with SquadStack ASR, where precision, efficiency, and affordability harmoniously cater to the needs of tech-savvy users and businesses alike.

This sample notebook shows you how to deploy [SquadStack Speech Recognition (Hinglish)]() using Amazon SageMaker.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

## Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to [SquadStack Speech Recognition (Hinglish)](). If so, skip step: [Subscribe to the model package](#1.-Subscribe-to-the-model-package)

## Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Create input payload](#B.-Create-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
   4. [Visualize output](#D.-Visualize-output)
   5. [Delete the endpoint](#E.-Delete-the-endpoint)
3. [Perform batch inference](#3.-Perform-batch-inference) 
4. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    2. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))
    

## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page [SquadStack Speech Recognition (Hinglish)]()
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

In [1]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [2]:
import os
import json
import uuid
import boto3
import base64

import numpy as np
import sagemaker as sage

from sagemaker import ModelPackage
from sagemaker import get_execution_role

In [3]:
role = get_execution_role()

sagemaker_session = sage.Session()

bucket = sagemaker_session.default_bucket()
runtime = boto3.client("runtime.sagemaker")
bucket

'sagemaker-ap-south-1-851467480473'

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [4]:
model_name = "asr-hinglish-telesales"

content_type = "application/x-recordio-protobuf"

real_time_inference_instance_type = (
    "ml.g4dn.xlarge"
)
batch_transform_inference_instance_type = (
    "ml.c5.2xlarge"
)

### A. Create an endpoint

In [5]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

--------!

Once endpoint has been created, you would be able to perform real-time inference.

### B. Create input payload

In [6]:
input_path = 'data/input/real-time/'
input_file_name = 'hinglish-example.mp3'

file_name = os.path.join(input_path, input_file_name)
file_name

'data/input/real-time/hinglish-example.mp3'

<Add code snippet that shows the payload contents>

In [7]:
output_path = 'data/output/real-time/'
output_file_name = os.path.join(
    output_path, 
    input_file_name.replace('.wav', '_out.json').replace('.mp3', '_out.json')
)

output_file_name

'data/output/real-time/hinglish-example_out.json'

### C. Perform real-time inference

In [8]:
!aws sagemaker-runtime invoke-endpoint --endpoint-name $model_name --body fileb://$file_name --content-type $content_type --region $sagemaker_session.boto_region_name $output_file_name

{
    "ContentType": "application/json",
    "InvokedProductionVariant": "AllTraffic"
}


### D. Visualize output

In [9]:
with open(output_file_name, 'r') as f:
    output = json.load(f)
    
output

{'metadata': {'channels': 2,
  'created_at': '2023-05-12 11:13:45.321490',
  'device': 'cuda:0',
  'model_version': 'ASR_telecommunication-hinglish',
  'request_id': 'baa38b74-f867-47f4-acd8-01cb4272bbcc',
  'total_duration_sec': 114.4795,
  'total_runtime_sec': 24.592},
 'results': {'transcription': {'0': 'Hello, good afternoon, sir. मैं देख पा रही हूं, आप हमारे website पर थे, sir, खुद का mobile application create करने के लिए. जी, क्या आप कोई coaching institute youtube channel चला रहे हैं या फिर student को कहीं पढ़ा रहे हैं? Sir? आपने बात किया है, sir, already? Ok, sir, क्या मैं जान सकती हूं, आपको like सारे details बताए गए थे, क्या, ok, sir, आप demo, इसके लिए demo भी दिखाते हैं, sir? तो क्या आप demo देखना चाहते हैं, sir? Ok, sir, मैं आपको कब call करूं, sir, confirm करने के लिए. Ok, ठीक है, sir, मैं आपको कल दुबारा call करती हूं, sir, आपका समय देने के लिए. धन्यवाद, sir.',
   '1': 'हां, हां, हां, हां, हां, बात हो गई है. हां, हां, हां, हां, detail बता दिया गया है. अभी decide नहीं किया है.

In [10]:
print ("Total channels: {}".format(output['metadata']['channels']))
print ("Total duration of call: {} sec".format(output['metadata']['total_duration_sec']))
print ("Total prediction time: {} sec, on device - {}".format(
    output['metadata']['total_runtime_sec'], output['metadata']['device']
))

Total channels: 2
Total duration of call: 114.4795 sec
Total prediction time: 24.592 sec, on device - cuda:0


In [11]:
for k, v in output['results']['transcription'].items():
    print ('\nChannel {}: "{}"\n'.format(k, v))


Channel 0: "Hello, good afternoon, sir. मैं देख पा रही हूं, आप हमारे website पर थे, sir, खुद का mobile application create करने के लिए. जी, क्या आप कोई coaching institute youtube channel चला रहे हैं या फिर student को कहीं पढ़ा रहे हैं? Sir? आपने बात किया है, sir, already? Ok, sir, क्या मैं जान सकती हूं, आपको like सारे details बताए गए थे, क्या, ok, sir, आप demo, इसके लिए demo भी दिखाते हैं, sir? तो क्या आप demo देखना चाहते हैं, sir? Ok, sir, मैं आपको कब call करूं, sir, confirm करने के लिए. Ok, ठीक है, sir, मैं आपको कल दुबारा call करती हूं, sir, आपका समय देने के लिए. धन्यवाद, sir."


Channel 1: "हां, हां, हां, हां, हां, बात हो गई है. हां, हां, हां, हां, detail बता दिया गया है. अभी decide नहीं किया है. नहीं, अभी हमको नहीं कुछ नहीं देखना. ठीक है, ठीक है."



### E. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [12]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 3. Perform batch inference

In this section, you will perform batch inference using multiple input payloads together. If you are not familiar with batch transform, and want to learn more, see these links:
1. [How it works](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-batch-transform.html)
2. [How to run a batch transform job](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html)

In [13]:
# upload the batch-transform job input files to S3
transform_input_folder = "data/input/batch"
transform_input = sagemaker_session.upload_data(transform_input_folder, key_prefix=model_name)
print("Transform input uploaded to " + transform_input)

Transform input uploaded to s3://sagemaker-ap-south-1-851467480473/asr-hinglish-telesales


In [14]:
# Run the batch-transform job
transformer = model.transformer(1, batch_transform_inference_instance_type)
transformer.transform(transform_input, content_type=content_type)
transformer.wait()

INFO:sagemaker:Creating transform job with name: asr-hinglish-telesales-2023-05-12-11-13-47-246


.................................................torchvision is not available - cannot save figures
 * Serving Flask app 'api'
 * Debug mode: off
#033#033WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.#033
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://169.254.255.131:8080
#033Press CTRL+C to quit#033
torchvision is not available - cannot save figures
 * Serving Flask app 'api'
 * Debug mode: off
#033#033WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.#033
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://169.254.255.131:8080
#033Press CTRL+C to quit#033
169.254.255.130 - - [12/May/2023 11:21:50] "GET /ping HTTP/1.1" 200 -
169.254.255.130 - - [12/May/2023 11:21:50] "#033GET /execution-parameters HTTP/1.1#033" 404 -
169.254.255.130 - - [12/May/2023 11:21:50

In [15]:
# output is available on following path
my_bucket = transformer.output_path.split('/')[2]
output_folder = transformer.output_path.split('/')[-1]
output_path = 'data/output/batch/'

transformer.output_path, bucket, output_folder, output_path

('s3://sagemaker-ap-south-1-851467480473/asr-hinglish-telesales-2023-05-12-11-13-47-246',
 'sagemaker-ap-south-1-851467480473',
 'asr-hinglish-telesales-2023-05-12-11-13-47-246',
 'data/output/batch/')

In [16]:
conn = boto3.client('s3')
contents = conn.list_objects(Bucket=my_bucket)

for content in contents['Contents']:

    if output_folder in str(content):
        
        response = conn.get_object(Bucket=my_bucket, Key=content['Key'])
        output = json.loads(response['Body'].read())
                
        output_file_name = os.path.join(output_path, content['Key'].split('/')[-1])
        
        with open(output_file_name, 'w') as f:
            json.dump(output, f, indent=4)
        
        print ("\n\nS3 Path: {}".format(os.path.join('s3://', my_bucket, content['Key'])))
        print ("\nTotal channels: {}".format(output['metadata']['channels']))
        print ("Total duration of call: {} sec".format(output['metadata']['total_duration_sec']))
        print ("Total prediction time: {} sec, on device - {}".format(
            output['metadata']['total_runtime_sec'], output['metadata']['device']
        ))
        
        for k, v in output['results']['transcription'].items():
            print ('\nChannel {} - Telesales Expert: "{}"\n'.format(k, v))



S3 Path: s3://sagemaker-ap-south-1-851467480473/asr-hinglish-telesales-2023-05-12-11-13-47-246/english-example.wav.out

Total channels: 1
Total duration of call: 27.0 sec
Total prediction time: 38.9744 sec, on device - cpu

Channel 0 - Telesales Expert: "Today, like this day, i will go to give after some sometime, and after coming from, i will have my you know. And after having my you know, i go out outside of house and play with trade of there, just like my brother actually. And then, after coming to home, i will go to."



S3 Path: s3://sagemaker-ap-south-1-851467480473/asr-hinglish-telesales-2023-05-12-11-13-47-246/hinglish-example.mp3.out

Total channels: 2
Total duration of call: 114.4795 sec
Total prediction time: 107.9683 sec, on device - cpu

Channel 0 - Telesales Expert: "Hello, good afternoon, sir. मैं देख पा रही हूं, आप हमारे website पर थे, sir, खुद का mobile application create करने के लिए. जी, क्या आप कोई coaching institute youtube channel चला रहे हैं या फिर student को कह

## 4. Clean-up

### A. Delete the model

In [17]:
model.delete_model()

INFO:sagemaker:Deleting model with name: asr-hinglish-telesales-2023-05-12-11-13-46-586


### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

